In [ ]:
import Pkg
Pkg.add("JuMP")
import Pkg; Pkg.add("JSON")
using JSON
import Pkg
Pkg.add("HiGHS")

In [ ]:
# Vs×S
x_values = []

# (E0×Q0)∪(Es×Qs)
y_value = []

# Et
z_values = []# Get Functions

function get_parameters(name_json)
    general_parameters = name_json["general_parameters"]
    return general_parameters
end

function get_wind_turbines(name_json)
    get_wind_turbines = name_json["wind_turbines"]

    return get_wind_turbines
end

function get_wind_scenarios(name_json)
    get_wind_scenarios = name_json["wind_scenarios"]
    return get_wind_scenarios
end

function get_substation_locations(name_json)
    get_substation_locations = name_json["substation_locations"]
    
    return get_substation_locations
end

function get_substation_types(name_json)
    get_substation_types = name_json["substation_types"]

    return get_substation_types
end

function get_land_substation_cable_types(name_json)
    get_land_substation_cable_types = name_json["land_substation_cable_types"]

    return get_land_substation_cable_types
end

function get_substation_substation_cable_types(name_json)
    get_substation_substation_cable_types = name_json["substation_substation_cable_types"]

    return get_substation_substation_cable_types
end

In [ ]:
#collectes des données utiles

file = json_small

P_max = get_parameters(file)["maximum_power"]
coords_mls = get_parameters(file)["main_land_station"]
C_ft = get_parameters(file)["fixed_cost_cable"]
C_lt = get_parameters(file)["variable_cost_cable"]
C_0 = get_parameters(file)["curtailing_cost"]
C_p = get_parameters(file)["curtailing_penalty"]
C_max = get_parameters(file)["maximum_curtailing"]
general_parameters = Dict("maximum_power" => P_max, "main_land_station" => coords_mls, "fixed_cost_cable" => C_ft, "variable_cost_cable" => C_lt, "curtailing_cost" => C_0, "curtailing_penalty" => C_p, "maximum_curtailing" => C_max)

wind_turbines = get_wind_turbines(file)
x_turbines = [turbine["x"] for turbine in wind_turbines]
y_turbines = [turbine["y"] for turbine in wind_turbines]
V_t = [x_turbines y_turbines]

wind_scenarios = get_wind_scenarios(file)

substation_locations = get_substation_locations(file)
x_substations = [substation["x"] for substation in substation_locations]
y_substations = [substation["y"] for substation in substation_locations]
V_s = [x_substations y_substations]

substation_types = get_substation_types(file)

land_substation_cable_types = get_land_substation_cable_types(file)

sub_sub_cable_types = get_substation_substation_cable_types(file)

In [11]:
using HiGHS
using JuMP

# Create a JuMP model with HiGHS solver
model = Model(HiGHS.Optimizer)

# Define binary variables 
@variable(model, x[x_values], binary=true)
@variable(model, y[y_values], binary=true)
@variable(model, z[z_values], binary=true)

# Set objective function to maximize
@objective(model, Max, 3x + 2y)

# Add constraints
@constraint(model, constraint1, 2x + y <= 10)
@constraint(model, constraint2, x + 3y <= 12)

# Solve the MILP problem
optimize!(model)

# Check if the optimization was successful
if termination_status(model) == MOI.OPTIMAL
    # Access solution information
    obj_value = objective_value(model)
    x_value = value(x)
    y_value = value(y)

    println("Objective Value: $obj_value")
    println("Optimal x Value: $x_value")
    println("Optimal y Value: $y_value")
else
    println("No optimal solution found.")
end


Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   4 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   23              -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   23              13                76.92%        0      0      0         0     0.0s
 R       0       0         0   0.00%   16.4            16                 2.50%        0      0      0         2     0.0s

Solving report
  Status            Optimal
  Primal bound      16
  Dual bound        16
  